In [ ]:
import pandas as pd
import numpy as np
import sys # Para la barra de progreso

def imputar_modificado(archivo_csv_entrada, archivo_csv_salida):
    """
    Imputa valores faltantes (NaN) en un archivo CSV de series de tiempo.

    Los NaN en cada columna (estación) se rellenan utilizando el promedio
    de los valores existentes para esa misma hora del día en esa misma estación.
    Esta función está adaptada del código original proporcionado.

    Args:
        archivo_csv_entrada (str): Nombre del archivo CSV de entrada (ej: 'O3.CSV').
        archivo_csv_salida (str): Nombre del archivo CSV de salida (ej: 'llenado_O3.CSV').
    """
    print(f'\nProcesando {archivo_csv_entrada}...')

    try:
        # 1. Leer el CSV, especificando la primera columna como índice,
        #    PERO SIN intentar parsear fechas automáticamente aquí.
        #    El índice será inicialmente de tipo 'Index' conteniendo strings.
        data = pd.read_csv(archivo_csv_entrada, index_col=0)
        print("Archivo leído inicialmente.")
        print(f"Tipo de índice ANTES de conversión: {type(data.index)}")
        # Opcional: Muestra las primeras filas del índice para verificar visualmente
        # print("Primeras 5 entradas del índice (como strings):\n", data.index[:5])

        # 2. Definir el formato esperado de las fechas/horas en el archivo CSV
        #    Basado en tu ejemplo '1/01/2013 0:00'
        formato_fecha_esperado = '%d/%m/%Y %H:%M'
        print(f"Intentando convertir el índice a Datetime usando el formato: '{formato_fecha_esperado}'")

        # 3. Convertir explícitamente el índice (que contiene los strings de 'FECHA') a DatetimeIndex
        #    errors='coerce': Si alguna cadena no coincide EXACTAMENTE con el formato,
        #                     se convertirá en NaT (Not a Time) en lugar de detener el script.
        original_index = data.index # Guardar una copia por si acaso
        data.index = pd.to_datetime(data.index, format=formato_fecha_esperado, errors='coerce')

        print(f"Tipo de índice DESPUÉS de conversión: {type(data.index)}")

        # 4. Verificar si la conversión fue exitosa y si hubo errores (NaT)
        if isinstance(data.index, pd.DatetimeIndex):
            print("Conversión a DatetimeIndex realizada.")
            nans_en_indice = data.index.isna().sum()
            if nans_en_indice > 0:
                print(f"\n¡ADVERTENCIA IMPORTANTE!: Se encontraron {nans_en_indice} fechas/horas en la columna 'FECHA'")
                print(f"que NO pudieron ser interpretadas usando el formato '{formato_fecha_esperado}'.")
                print("Estas entradas se han convertido a 'NaT' (Not a Time) en el índice.")
                print("Esto puede afectar los cálculos y la imputación.")
                print("-> REVISA tu archivo CSV original en las filas correspondientes a los NaT para corregir el formato.")
                # Podrías querer ver cuáles fallaron (esto puede ser largo si hay muchas):
                # print("Índices originales que fallaron la conversión:")
                # print(original_index[data.index.isna()])
                # Decide si continuar o detenerte. Por ahora, continuaremos pero los NaT no se imputarán bien.
            else:
                print("Todas las fechas/horas del índice se convirtieron correctamente.")

        else:
            # Si la conversión falló completamente y el índice NO es DatetimeIndex
            print("\nError CRÍTICO: La conversión explícita a DatetimeIndex falló.")
            print(f"El índice sigue siendo de tipo: {type(data.index)}")
            print("Asegúrate de que TODAS las entradas en la primera columna ('FECHA')")
            print(f"sigan EXACTAMENTE el formato '{formato_fecha_esperado}' (ej: '1/01/2013 0:00').")
            print("Incluso un solo valor con formato diferente puede causar este fallo si 'errors' no es 'coerce'.")
            return # Detener la ejecución

        # --- Si llegamos aquí, data.index ES un DatetimeIndex (aunque pueda tener NaTs) ---

        print(f"\nColumnas encontradas: {data.columns.tolist()}")
        print(f"Dimensiones originales: {data.shape}")
        # Recalcular NaNs por si la conversión a NaT afectó filas enteras (poco probable aquí)
        print(f"Valores faltantes ANTES de imputar (por columna):\n{data.isnull().sum()}")

        # Calcular promedios horarios (esto ignorará filas con índice NaT si las hubiera)
        # Usamos .dt.hour que funciona sobre DatetimeIndex
        promedios_horas = data.groupby(data.index.dt.hour).mean()
        print("\nPromedios horarios calculados.")
        # print(promedios_horas)

        data_imputada = data.copy()

        print("Iniciando imputación por hora (puede tardar)...")
        suma = 0
        num_columnas = len(data.columns)
        estaciones_procesadas = 0
        imputaciones_realizadas = 0

        for name_estacion in data.columns:
            estaciones_procesadas += 1
            progress = int(estaciones_procesadas / num_columnas * 100)
            sys.stdout.write('\r')
            sys.stdout.write('[%-20s] %d%% Procesando: %s' % ('=' * int(progress/5), progress, name_estacion))
            sys.stdout.flush()

            for indice_tiempo in data_imputada.index:
                 # Saltar si el índice mismo es NaT (fecha inválida)
                 if pd.isna(indice_tiempo):
                     continue

                 # Verificar si el valor de datos es NaN
                 if pd.isna(data_imputada.loc[indice_tiempo, name_estacion]):
                    hora_int = indice_tiempo.hour # Acceder a la hora (entero)

                    if hora_int in promedios_horas.index:
                       valor_imputado = promedios_horas.loc[hora_int, name_estacion]
                       if not pd.isna(valor_imputado):
                           data_imputada.loc[indice_tiempo, name_estacion] = valor_imputado
                           imputaciones_realizadas += 1

        sys.stdout.write('\n')
        print(f"Imputación por hora completada. Se realizaron {imputaciones_realizadas} imputaciones.")

        nans_restantes = data_imputada.isnull().sum()
        total_nans_restantes = nans_restantes.sum()
        if total_nans_restantes > 0:
             print(f"\nAdvertencia: Quedaron {total_nans_restantes} valores NaN después de la imputación.")
             # Explicar posible causa de NaNs restantes
             print("Causas posibles: (1) Todos los valores originales para una combinación estación/hora eran NaN.")
             print("                  (2) Filas donde la fecha original era inválida (índice NaT) no fueron imputadas.")
             print(f"NaNs restantes por columna:\n{nans_restantes[nans_restantes > 0]}")
        else:
             print("\nNo quedaron valores NaN después de la imputación horaria.")

        # Guardar el resultado
        # Nota: Las filas con índice NaT (si las hubo) se guardarán con una entrada vacía
        # o 'NaT' en la columna de índice del archivo CSV de salida.
        data_imputada.to_csv(archivo_csv_salida)
        print(f'\nProceso completado. Archivo imputado guardado como: {archivo_csv_salida}')

    except FileNotFoundError:
        print(f"Error Fatal: El archivo '{archivo_csv_entrada}' no se encontró.")
        print("Por favor, asegúrate de que 'O3.CSV' esté en el mismo directorio que este script.")
    except ValueError as e:
        # Captura errores si pd.to_datetime falla y no usamos errors='coerce'
        print(f"\nError de Valor durante la conversión de fecha/hora: {e}")
        print("Esto generalmente significa que una o más cadenas en la columna de índice")
        print(f"no coinciden con el formato esperado: '{formato_fecha_esperado}'.")
    except KeyError as e:
         print(f"\nError de Clave: {e}.")
         print("Puede que una columna esperada no exista o falte la columna índice.")
         print("Asegúrate de que la primera columna es 'FECHA'.")
    except Exception as e:
        print(f"\nOcurrió un error inesperado durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()

# --- Bloque de Ejecución Principal ---
if __name__ == "__main__":
    nombre_archivo_entrada = 'O3.csv'
    nombre_archivo_salida = 'llenado_O3.CSV'
    imputar_modificado(nombre_archivo_entrada, nombre_archivo_salida)


Procesando O3.csv...
Archivo leído inicialmente.
Tipo de índice ANTES de conversión: <class 'pandas.core.indexes.base.Index'>
Intentando convertir el índice a Datetime usando el formato: '%d/%m/%Y %H:%M'
Tipo de índice DESPUÉS de conversión: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Conversión a DatetimeIndex realizada.
Todas las fechas/horas del índice se convirtieron correctamente.

Columnas encontradas: ['BOLIVIA', 'BOSA', 'CARVAJAL', 'CENTRO DE ALTO RENDIMIENTO', 'CIUDAD BOLIVAR', 'FONTIBON', 'GUAYMARAL', 'JAZMIN', 'KENNEDY', 'LAS FERIAS', 'MIN AMBIENTE', 'PUENTE ARANDA', 'SAN CRISTOBAL', 'SUBA', 'TUNAL', 'USAQUEN', 'USME', 'COLINA', 'MOVIL FONTIBON']
Dimensiones originales: (87648, 19)
Valores faltantes ANTES de imputar (por columna):
BOLIVIA                       76368
BOSA                          85879
CARVAJAL                      20767
CENTRO DE ALTO RENDIMIENTO    23194
CIUDAD BOLIVAR                70029
FONTIBON                      56290
GUAYMARAL            

Traceback (most recent call last):
  File "<ipython-input-17-3f5b21b843f6>", line 77, in imputar_modificado
    promedios_horas = data.groupby(data.index.dt.hour).mean()
                                   ^^^^^^^^^^^^^
AttributeError: 'DatetimeIndex' object has no attribute 'dt'


In [ ]:
#!/usr/bin/env python
# coding: utf-8

from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os

In [ ]:
import pandas as pd
import numpy as np
import sys

def imputar(archivo_csv):
    print('\nProcesando', archivo_csv)

    # Leer el archivo sin prefijo de carpeta
    data = pd.read_csv(archivo_csv, index_col=0, parse_dates=True)
    data_nan = data.fillna(999)
    data["FECHA"] = pd.to_datetime(data["FECHA"], format="%d/%m/%Y %H:%M", errors="coerce")

    # Calcular promedios horarios
    promedios_horas = data.groupby(data.index.strftime('%H')).mean()

    suma = 0
    for name_estacion in data.columns:
        suma += 1
        progress = int(suma / len(data.columns) * 100)
        sys.stdout.write('\r')
        sys.stdout.write('[%-20s] %d%%' % ('=' * (progress // 5), progress))
        sys.stdout.flush()

        for i in range(len(data_nan[name_estacion])):
            if data_nan[name_estacion].iloc[i] == 999:
                hora = int(data_nan.index[i].hour)
                data_nan.at[data_nan.index[i], name_estacion] = promedios_horas[name_estacion][hora]

    # Guardar el resultado sin prefijo de carpeta
    output_file = 'llenado_' + archivo_csv
    data_nan.to_csv(output_file)
    print(f"\nArchivo guardado correctamente: {output_file}")

def mean_axis1(archivo_csv):
    data = pd.read_csv(archivo_csv, index_col=0, parse_dates=True)
    mean = data.mean(axis=1)
    mean[np.isnan(mean)] = np.nanmean(mean)

    mean_df = pd.DataFrame({'MEAN': mean})
    output_file = 'mean_' + archivo_csv
    mean_df.to_csv(output_file)
    print(f"\nArchivo guardado correctamente: {output_file}")

def change_dates(filee):
    df0 = pd.read_csv('O3.csv')
    df1 = pd.read_csv(filee)
    df1['FECHA'] = df0['FECHA']

    df1.to_csv(filee, index=False)
    print(f"\nArchivo modificado correctamente: {filee}")


In [ ]:
imputar('O3.csv')


Procesando O3.csv


KeyError: 'FECHA'

In [ ]:
def imputar(archivo_csv):
    print(''); print('Procesando', archivo_csv)
    data            = pd.read_csv('DATA_ORIGINAL/' + archivo_csv, index_col = 0, parse_dates=True)
    data_nan        = data.fillna(999)
    promedios_horas = data.groupby(data.index.strftime('%H')).mean()

    suma = 0
    for name_estacion in data.columns:
        suma += 1; progress = int(suma/len(data.columns) * 100)
        sys.stdout.write('\r')
        sys.stdout.write('[%-20s] %d%%' % ('=' * progress, progress))
        sys.stdout.flush()

        for i in np.arange(0,len(data_nan[name_estacion]),1):
            dato = data_nan[name_estacion][i]

            if dato == 999:
                hora = int(data_nan[name_estacion].index[i].hour)
                data_nan[name_estacion][i] = promedios_horas[name_estacion][hora]
    # QUITAR esta linea para tener todas las estaciones
    #data_nan = pd.DataFrame(data_nan.mean(axis=1)).rename(columns={0: 'MEAN'})
    data_nan.to_csv('SALIDAS/llenado_' + archivo_csv)

def mean_axis1(archivo_csv):
    data = pd.read_csv('DATA_ORIGINAL/' + archivo_csv, index_col = 0, parse_dates=True)
    mean = data.mean(axis=1)
    mean = np.array(mean)
    inds = np.where(np.isnan(mean))
    mean[inds] = np.nanmean(mean)
    mean = pd.DataFrame({'MEAN':mean})
    mean.to_csv('SALIDAS/llenado_' + archivo_csv)


def change_dates(filee):
    df0 = pd.read_csv('DATA_ORIGINAL/NO.csv')
    df1 = pd.read_csv('DATA_ORIGINAL/'+filee)
    df1['FECHA'] = df0['FECHA']
    df1.to_csv('DATA_ORIGINAL/'+filee, index = False)

In [ ]:
Path('SALIDAS').mkdir(parents=True, exist_ok=True)
lista_files = os.listdir('DATA_ORIGINAL/')

chem = 'no'
if chem == 'yes':
    lista_files = ['VEL.csv']
    for i in lista_files:
        change_dates(i)
        imputar(i)

In [ ]:
data_ws = pd.read_csv('SALIDAS/llenado_VEL.csv')
change_dates('DIR.csv')
data_dir = pd.read_csv('DATA_ORIGINAL/DIR.csv',index_col=0,parse_dates=True)

wind = components(data_ws,data_dir)

<ipython-input-2-fd436de1f853>:47: RuntimeWarning: Mean of empty slice
  u_wind = np.nanmean(ut,axis=1)
<ipython-input-2-fd436de1f853>:50: RuntimeWarning: Mean of empty slice
  v_wind = np.nanmean(vt,axis=1)


In [ ]:
def components(ws,wd):

    ut = np.ones((len(wd),len(wd.columns)))
    vt = np.ones((len(wd),len(wd.columns)))

    for i,col in enumerate(wd.columns):
        #Convert to components!
        u_wind = np.array(ws['MEAN']) * np.sin(np.array(wd[col]) * np.pi/180)
        v_wind = np.array(ws['MEAN']) * np.cos(np.array(wd[col]) * np.pi/180)
        #Add the components to a matrix
        ut[:,i] = u_wind
        vt[:,i] = v_wind

    #Calculate the stations mean and replace the nan values!
    u_wind = np.nanmean(ut,axis=1)
    inds = np.where(np.isnan(u_wind))
    u_wind[inds] = np.nanmean(u_wind)
    v_wind = np.nanmean(vt,axis=1)
    inds = np.where(np.isnan(v_wind))
    v_wind[inds] = np.nanmean(v_wind)

    ## Add U
    df0 = pd.DataFrame(np.column_stack((ws['FECHA'],u_wind)), columns=['FECHA','MEAN'])
    df0.to_csv('SALIDAS/llenado_u-component.csv',index=False)

    ## Add V
    df1 = pd.DataFrame(np.column_stack((ws['FECHA'],v_wind)), columns=['FECHA','MEAN'])
    df1.to_csv('SALIDAS/llenado_v-component.csv',index=False)

    #Calculate the wind direction using the wind components!
    mean_wd = np.arctan2(u_wind, v_wind) * 180/np.pi
    #Result is from -180 to 180 due to arctan2, this move it between 0 to 360
    mean_wd = (180 + mean_wd)
    df2 = pd.DataFrame(np.column_stack((ws['FECHA'],mean_wd)), columns=['FECHA','MEAN'])
    df2.to_csv('SALIDAS/llenado_DIR.csv',index=False)
    return(mean_wd)